<a href="https://colab.research.google.com/github/pullz6/Retail_Prediction/blob/main/Retail_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


